In [1]:
import hashlib
import json
from typing import Annotated, Any

import requests
import uvicorn
from fastapi import Body, FastAPI, File, Form, Header, UploadFile
from fastapi.encoders import jsonable_encoder
from fastapi.responses import (
    HTMLResponse,
    ORJSONResponse,
    PlainTextResponse,
    RedirectResponse,
    Response,
    StreamingResponse,
)
from nest_asyncio import apply
from pydantic import BaseModel, EmailStr

In [ ]:
app = FastAPI()

@app.get('/',response_class=HTMLResponse)
async def index():
    html ="""
    <html>
    <body>
        <form action="/uploadfiles/" enctype="multipart/form-data" method="post">
            <input name="files" type="file" multiple>
            <input type="submit">
        </form>
    </body>
    </html>
    """
    return html
    
@app.post('/uploadfiles/')
async def upload(files : UploadFile = File()):
    contents = await files.read()
    await files.seek(0)

    import shutil

    with open(f"../temp/{files.filename}", "w+b") as f:
        shutil.copyfileobj(files.file, f)

    return {"message": "File are Uploaded"}
    
    
if __name__ == "__main__":
    apply()
    uvicorn.run(app, port=4444)

INFO:     Started server process [670641]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:49468 - "POST /uploadfiles/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:49468 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:51062 - "POST /uploadfiles/ HTTP/1.1" 200 OK
